In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import numpy
import scipy
import pandas 
states_scores = defaultdict(list)

def process_row(row):
    i = 1
    for word in row[1:]:
        if word is not nan and word.replace(',','').isnumeric():
            row[i] = int(word.replace(',',''))
        if word == '--' or word == '' or word == '*':
            row[i] = 0
        if word.find('.') != -1 and word not in ['Col.','Col.*']:
            if word.find('%') != -1:
                word = word[0:-1]
            row[i] = float(word)
        i += 1

    return row

def get_index_of_first_int(row):
    i = 0
    for word in row:
        if word.replace(',','').isnumeric():
            return i
        i += 1

    return i

def get_specific_election(year):
    return f"https://www.presidency.ucsb.edu/statistics/elections/{year}"

dataframe_list = []
for year in range(1900,2004,4):
    if year == 1976:
        continue
    driver_exe = 'chromedriver'
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(driver_exe, options=options)
    URL = get_specific_election(year)
    print(URL)
    driver.get(URL)
    a = driver.find_elements(By.XPATH, "//table[@class='table table-responsive']")
    results = a[0].text
    republican_indx = results.find('Republican')
    democratic_indx = results.find('Democratic')
    only_votes = results[results.find('Alabama'):]
    state_per_row = only_votes.split('\n')
    parse_rows = [row.split(' ') for row in state_per_row]
    for row in parse_rows:
        row = list(dict.fromkeys(row))
        indx = get_index_of_first_int(row)
        row = process_row(row)
        if republican_indx < democratic_indx:
            metrics = row[indx:]
        else:
            metrics = [row[indx]]+ row[indx+4:indx+7] + row[indx+1:indx+4] + row[indx+7:]
        state = row[0:indx]
        state = ' '.join(state)
        pattern = r'[^A-Za-z ]+'
        state = re.sub(pattern, '', state)
        metrics.insert(0, state)
        metrics_len = len(metrics)
        if metrics_len < 14:
            buffer = 14 - metrics_len
            buffer_lst = [nan] * buffer
            metrics += buffer_lst
        dataframe_list.append(metrics)
        if state == 'Wyoming':
            break

columns = [f'state',f'year'f'total_votes',f'rep_votes',f'rep_percent',f'rep_ev', f'dem_votes',f'dem_percent', f'dem_ev',
               f'other_votes', f'other_percent', f'other_ev',f'socialist_vote', f'socialist_percent', f'socialist_ev']

table = pandas.DataFrame(dataframe_list, columns=columns)
table.to_csv(r'Election_project1.csv', index = False, header=True)




https://www.presidency.ucsb.edu/statistics/elections/1900
https://www.presidency.ucsb.edu/statistics/elections/1904
https://www.presidency.ucsb.edu/statistics/elections/1908
https://www.presidency.ucsb.edu/statistics/elections/1912
https://www.presidency.ucsb.edu/statistics/elections/1916
https://www.presidency.ucsb.edu/statistics/elections/1920
https://www.presidency.ucsb.edu/statistics/elections/1924
https://www.presidency.ucsb.edu/statistics/elections/1928
https://www.presidency.ucsb.edu/statistics/elections/1932
https://www.presidency.ucsb.edu/statistics/elections/1936
https://www.presidency.ucsb.edu/statistics/elections/1940
https://www.presidency.ucsb.edu/statistics/elections/1944
https://www.presidency.ucsb.edu/statistics/elections/1948
https://www.presidency.ucsb.edu/statistics/elections/1952
https://www.presidency.ucsb.edu/statistics/elections/1956
https://www.presidency.ucsb.edu/statistics/elections/1960
https://www.presidency.ucsb.edu/statistics/elections/1964
https://www.pr

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   state              1221 non-null   object 
 1   total_votes        1221 non-null   int64  
 2   rep_votes          1221 non-null   int64  
 3   rep_percent        1221 non-null   float64
 4   rep_ev             1220 non-null   object 
 5   dem_votes          1220 non-null   float64
 6   dem_percent        1220 non-null   float64
 7   dem_ev             1216 non-null   object 
 8   other_votes        399 non-null    float64
 9   other_percent      397 non-null    float64
 10  other_ev           50 non-null     float64
 11  socialist_vote     48 non-null     float64
 12  socialist_percent  0 non-null      float64
 13  socialist_ev       0 non-null      float64
dtypes: float64(9), int64(2), object(3)
memory usage: 133.7+ KB


In [24]:
table

,state,total_votes,rep_votes,rep_percent,rep_ev,dem_votes,dem_percent,dem_ev,other_votes,other_percent,other_ev,socialist_vote,socialist_percent,socialist_ev
0,Alabama,160477,55634,34.7,0,96368.0,60.1,11,NaN,NaN,NaN,NaN,NaN,NaN
1,Arkansas,127966,44800,35.0,0,81242.0,63.5,8,NaN,NaN,NaN,NaN,NaN,NaN
2,California,302399,164755,54.5,9,124985.0,41.3,0,NaN,NaN,NaN,NaN,NaN,NaN
3,Colorado,220895,92701,42.0,0,122705.0,55.5,4,NaN,NaN,NaN,NaN,NaN,NaN
4,Connecticut,180195,102572,56.9,6,74014.0,41.1,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,Virginia,2739447,1437490,52.5,13,1217290.0,44.4,0,59398.0,2.2,NaN,NaN,NaN,NaN
1217,Washington,2487433,1108864,44.6,0,1247652.0,50.2,11,103002.0,4.1,NaN,NaN,NaN,NaN
1218,West Virginia,648124,336475,51.9,5,295497.0,45.6,0,10680.0,1.6,NaN,NaN,NaN,NaN
1219,Wisconsin,2598607,1237279,47.6,0,1242987.0,47.8,11,94070.0,3.6,NaN,NaN,NaN,NaN


In [25]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   state              1221 non-null   object 
 1   total_votes        1221 non-null   int64  
 2   rep_votes          1221 non-null   int64  
 3   rep_percent        1221 non-null   float64
 4   rep_ev             1220 non-null   object 
 5   dem_votes          1220 non-null   float64
 6   dem_percent        1220 non-null   float64
 7   dem_ev             1216 non-null   object 
 8   other_votes        399 non-null    float64
 9   other_percent      397 non-null    float64
 10  other_ev           50 non-null     float64
 11  socialist_vote     48 non-null     float64
 12  socialist_percent  0 non-null      float64
 13  socialist_ev       0 non-null      float64
dtypes: float64(9), int64(2), object(3)
memory usage: 133.7+ KB


In [26]:
table.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1216    False
1217    False
1218    False
1219    False
1220    False
Length: 1221, dtype: bool